# Análisis y búsqueda sobre un corpus de documentos científicos

Nos disponemos en este notebook a mostrar los pasos seguidos para la realización de la primera práctica de la asignatura, incluyendo los dos ejercicios opcionales sobre Gensim y Whoosh.

# Importación de librerías

Antes de comenzar con la extracción de artículos de *Google Scholar*, importamos todas las librerías que se van a usar a lo largo de la realización de la práctica.

In [1]:
# librería que nos permite extraer artículos científicos de Google Scholar
import scholarly

# librería necesaria para leer archivos de la web
from urllib.request import urlopen  

# importamos la librería que nos permite realizar operaciones con ficheros json, como lectura y escritura
import json

# funciones del parser PDFMiner que se necesitan
from pdfminer.pdfparser import PDFParser, PDFSyntaxError
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter

# funciones de entrada/salida necesarias en el parseado
from io import BytesIO
from io import StringIO

# funciones de scikit-learn necesarias para vectorizar la colección de documentos
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# librería que nos permite aplicar Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# librerías que usaremos durante el proceso de aplicación de Gensim
import re
import numpy as np
import pandas as pd
from pprint import pprint

# librería spacy para la "lemmatización" en Gensim
import spacy

# para descargar stopwords disponibles en la librería nltk
from nltk.corpus import stopwords

# para poder obtener las stopwords de nltk, hemos de ejecutar la siguiente línea, la cual mostramos comentada porque ya las tenemos descargadas
# nltk.download('stopwords')

# herramientas de plotting para gensim
import pyLDAvis
import pyLDAvis.gensim

# librerías y funciones usadas para aplicar Whoosh
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StandardAnalyzer
from whoosh import index
from whoosh.qparser import QueryParser


# Extracción de artículos

En esta sección vamos a mostrar los pasos 1, 2 y 3 pedidos en el enunciado de la práctica sobre la obtención de artículos científicos de *Google Scholar* y su posterior almacenamiento en disco.

El primer paso es utilizar la librería *scholarly* para obtener artículos científicos de una serie de tópicos. Para ello hemos usado la función *search_pubs_query*, que devuelve un generador con objetos estilo *json* que tienen la siguiente estructura:

- **_filled**
- **bib**
   - abstract, e.g.: "420"
   - author, e.g.: "Antonio López"
   - eprint, e.g.: "http://sigitwidiyanto.staff.gunadarma.ac.id/Downloads/files/38034/M8-Note-kMeans.pdf"
   - title, e.g.: "K-means clustering tutorial"
   - url, e.g.: "http://sigitwidiyanto.staff.gunadarma.ac.id/Downloads/files/38034/M8-Note-kMeans.pdf"
- **source**
- **citedby** 
<br>
...

La información que nos interesa extraer se encuentra dentro del campo *bib*. En concreto, vamos a extraer el título, el autor, el abstract y la url del artículo. <br>
Examinando los resultados de obtener artículos mediante esta función, hemos observado que hay artículos que tienen la url en los campos *eprint* y *url*, otros solamente en *url* o *eprint*, y otros en ninguno. Por ello, cuando procesemos la información de los artículos, realizaremos un análisis de casos para quedarnos con el elemento que tenga la url, si existe.

Antes de realizar las llamadas a la función de *scholarly* con los diferentes tópicos, vamos a definir una función que nos permite *parsear* y extraer el texto de un archivo pdf existente en una página web. La primera librería que se intentó usar para este *parseado* fue PyPDF2. Sin embargo, esta tiene una limitación muy grande: devuelve todos los caracteres juntos salvo los espacios, lo que hace que el texto sea ilegible. Por ello decidió usarse la librería PDFMiner, con la que se consiguió *parsear* los PDFs de manera correcta.

La función creada utiliza la librería *urllib* para leer el PDF y a continuación *parsea* el fichero y va procesando página a página el documento para guardar cada una de las líneas del fichero en una lista de strings. El último paso es aplicar la función *join* para concatenar todos los elementos de la lista en un mismo string cuyas líneas estén separadas por saltos de línea.

A lo largo de la función se han ido añadiendo bloques *try - except* para devolver cadenas vacías en caso de que ocurra un error al leer y extraer el fichero PDF. Algunos de esos errores que hemos percibido son que el artículo ya no existe, que hay que pasar un certificado para obtener el artículo, o que aunque la extensión sea .pdf el archivo no es realmente un pdf.

In [2]:
def pdf_url2txt(url):
        
    # para evitar errores como que el archivo ya no exista o que haya que pasar un certificado
    try:
        remoteFile = urlopen(url).read()
    except:
        return ''

    # stream que recibe los bytes devueltos por la función read
    memoryFile = BytesIO(remoteFile)
    parser = PDFParser(memoryFile)
    
    # si no se puede extraer el documento, devolvemos la cadena vacía
    try:
        document = PDFDocument(parser)
    except: 
        return ''
    
    if not document.is_extractable:
        return ''
    
    rsrcmgr = PDFResourceManager()
    
    # creamos un buffer para el texto parseado
    retstr = StringIO()
    
    # parámetros de espaciado para el parsing
    laparams = LAParams()
    codec = 'utf-8'
     
    device = TextConverter(rsrcmgr, retstr, 
                           codec = codec, 
                           laparams = laparams)
    
    # Creamos un objeto de intérprete de PDFs
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # Procesamos cada página del documento
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
     
    # Guardamos cada línea en una lista
    records = []
    lines = retstr.getvalue().splitlines()
    for line in lines:
        records.append(line)
    
    # Concatenamos los elementos de la lista en un string final
    rec = '\n'.join(records)
    return rec

Una vez creada la función para *parsear* los PDFs, procedemos a aplicar la función de búsqueda de *scholarly* para 5 tópicos distintos. En ellos hemos mezclado conceptos de redes, análisis complejo, procesamiento de imágenes, aprendizaje no supervisado y bases de datos. El objetivo es observar si cuando apliquemos Gensim y obtengamos 5 tópicos, este los separa en los 5 grupos que introducimos aquí. 

Para cada uno de esos tópicos, comprobamos si tiene una url que acaba en '.pdf' o no. Si conseguimos parsearla, se guarda en una variable *text*, mientras que si no podemos, se asigna a *text* la cadena vacía. De igual manera ocurre con el abstract. Una vez tenemos todos los campos mencionados anteriormente, los añadimos a una lista de diccionarios que guardan la información de cada uno de los artículos, sean del tópico que sean.

Hemos decidido que el número de artículos descargados para cada tópico sea de 30, lo que nos da un total de 150 artículos. Para este fin hemos creado una variable *cont* que cuenta el número de iteraciones por artículo y para el bucle cuando se llega a 30.

In [3]:
# elegimos los tópicos
topics = ["cauchy-riemann equations", "image processing", "eduroam internet", "k-mean clustering", "NoSQL databases"]
data_articles = []

# para cada tópico, extraemos 30 artículos, añadiendo a la lista data_articles sus características
for topic in topics:
    pubs = scholarly.search_pubs_query(topic)
    cont = 0
    for pub in pubs:
        
        # añadimos el texto si existe un pdf accesible que contenga el artículo
        text = ''
        if 'url' in pub.bib:
            if pub.bib['url'].endswith('.pdf'):
                text = pdf_url2txt(pub.bib['url'])
        elif text == '' and 'eprint' in pub.bib:
            if pub.bib['eprint'].endswith('.pdf'):
                text = pdf_url2txt(pub.bib['eprint'])

        # añadimos el abstract si está disponible
        abstract = ''
        if 'abstract' in pub.bib:
            abstract = pub.bib['abstract']

        # añadimos los campos de cada artículo como un diccionario a la lista data_articles
        data_articles.append({
            'text': text,
            'abstract': abstract,
            'title': pub.bib['title'],
            'author': pub.bib['author']
        })
        
        cont += 1
        if cont == 30: 
            break

Para cumplir con el apartado 3 de la práctica, hemos decidido guardar los datos en disco en un formato json, usando la función *json.dump*. Lo que hace el siguiente código es: 
1. Trata de abrir el fichero pubs.json existente en nuestro directorio actual, para escritura.
2. Escribe la anterior lista en el fichero utilizando usando como nivel de indentación 3, para que la salida sea más legible.
3. La sentencia *with* se encarga de cerrar el fichero una vez terminemos de escribir.

In [4]:
with open('pubs.json', 'w') as json_file:
    json.dump(data_articles, json_file, indent=3)

# Procesamiento del corpus para la vectorización

En esta sección vamos a realizar el apartado 4 de la tarea. Primero, para formar la colección de artículos que se va a vectorizar, vamos a crear una lista que contenga, si existe, el texto de los artículos. En su defecto, se usará el abstract. Y si tampoco tiene abstract, añadimos el título:

In [5]:
articles = []
for article in data_articles:
    if article["text"] != "":
        articles.append(article["text"])
    elif article["abstract"] != "":
        articles.append(article["abstract"])
    else:
        articles.append(article["title"])

Para la realización de este apartado, siguiendo las indicaciones del enunciado de la práctica, hemos usado la librería *scikit-learn*. Comenzamos con el cálculo de los 50 términos más "centrales" de la colección, usando el TfIdfVectorizer. 

El primer paso es crear un objeto TfIdfVectorizer al que le imponemos, usando distintos parámetros:
- Que las *stopwords* sean en inglés, ya que hemos buscado los artículos en inglés.
- Que queremos únicamente las palabras que aparezcan como mínimo en 10 documentos. Esto se realiza mediante el parámetro min_df, que puede ser de dos tipos: float entre 0 y 1, que indica la proporción de documentos en los que tienen que aparecer los términos, o entero, que indica el número mínimo de documentos en los que queremos que aparezcan. En nuestro caso, lo hemos usado como float, y le hemos asignado el valor 10/total de documentos (sería lo mismo que poner min_df = 10).
- Una expresión regular que hace que solo se muestren palabras con letras (no números), y que contengan como mínimo dos caracteres.

Una vez creado el objeto, se utiliza la función *fit_transform* para devolver una matriz que contiene el TF-IDF de cada término (representado como un entero) en cada documento. Para obtener lo que nos pide el enunciado, hemos de obtener la suma de los tf-idf a lo largo de la colección para cada término. Primero, convertimos la matriz obtenida en el paso anterior en un array de arrays, y una vez lo tenemos sumamos todos los arrays para obtener un único vector que contiene la suma para cada palabra. Esta suma la hemos realizado con *numpy*, que de forma sencilla nos permite aplicar operaciones a los arrays en cualquiera de sus ejes. En este caso, diciéndole que queremos sumar el eje 0, suma cada uno de los elementos del array principal, es decir, cada uno de los arrays contenidos en él.

In [6]:
vectorizer_center = TfidfVectorizer(stop_words='english', min_df = 10/len(data_articles), token_pattern=r'(?u)\b[A-Za-z][A-Za-z]+\b')
tfidf_matrix = vectorizer_center.fit_transform(articles)
# calculamos la suma del tf idf a lo largo de todos los artículos
tfidf_sum = np.sum(tfidf_matrix.toarray(), axis = 0)

Una vez aplicados los pasos anteriores, ya tenemos en un único array la suma de los TF-IDF de cada palabra a lo largo de la colección. Como hemos indicado, la función *fit_transform* nos ha devuelto cada palabra como un número, pero nosotros queremos saber cuáles son esas palabras. Utilizando la función *get_feature_names* las obtenemos. 

Ya solo queda un paso, obtener las palabras con mayor TF-IDF. Para ello, dado que los índices de ambos vectores (*tfidf_sum* y *tfidf_names*) hacen referencia a los mismos términos, únicamente tenemos que ordenar el primer vector en orden descendente y reordenar el segundo en base al primero. Para ello hemos usado la función *zip*, que devuelve pares de elementos de dos listas distintas, y sobre ese objeto *zip* aplicamos la función *sorted*, la cual, si no se le indica lo contrario, ordena la colección de tuplas en base al primer elemento de las tuplas. Si de esa colección ordenada extraemos el vector con los segundos elementos de las tuplas, obtenemos los nombres ordenados por valor de TF-IDF. Por último, extraemos los 50 primeros términos de ese vector y ya tenemos los 50 términos más "centrales" de la colección.

In [7]:
# obtenemos las palabras devueltas por el vectorizer
tfidf_names = vectorizer_center.get_feature_names()
# ordenamos la lista de la puntuación tfidf y obtenemos los nombres asociados a cada puntuación usando la función zip, y nos quedamos
# con las 50 palabras con la puntuación más alta.
centered_names = [x for _,x in sorted(zip(tfidf_sum,tfidf_names), reverse = True)][:50]

Las 50 palabras más centrales obtenidas son las siguientes:

In [8]:
centered_names

['image',
 'data',
 'eduroam',
 'processing',
 'databases',
 'nosql',
 'equations',
 'cauchy',
 'internet',
 'clustering',
 'riemann',
 'digital',
 'algorithm',
 'access',
 'la',
 'used',
 'database',
 'relational',
 'en',
 'segmentation',
 'paper',
 'using',
 'images',
 'mean',
 'complex',
 'computer',
 'based',
 'book',
 'systems',
 'cluster',
 'applications',
 'large',
 'network',
 'real',
 'number',
 'institutions',
 'user',
 'new',
 'analysis',
 'radius',
 'problem',
 'users',
 'use',
 'roaming',
 'order',
 'clusters',
 'vision',
 'institution',
 'different',
 'function']

Por otro lado, para obtener los 100 términos más repetidos en la colección, aplicamos el mismo procedimiento que en el paso anterior pero usando el *CountVectorizer*, que nos devuelve el número de veces que aparece cada término en cada documento, es decir, la frecuencia de ese término (TF) en cada documento.

Una vez tenemos estas frecuencias, de igual manera que en el caso anterior, procedemos a sumar los arrays y ordenamos el resultado para obtener las 100 palabras más repetidas.

In [9]:
# para mostrar los 100 términos más repetidos usamos el CountVectorizer
vectorizer_count = CountVectorizer(stop_words='english', min_df = 10/len(data_articles), token_pattern=r'(?u)\b[A-Za-z][A-Za-z]+\b')
count_matrix = vectorizer_count.fit_transform(articles)

# calculamos la suma del tf a lo largo de todos los artículos
count_sum = np.sum(count_matrix.toarray(), axis = 0)

# obtenemos las palabras devueltas por el vectorizer
count_names = vectorizer_count.get_feature_names()
repeated_names = [x for _,x in sorted(zip(count_sum,count_names), reverse = True)][:100]

Las 100 palabras obtenidas son las siguientes:

In [10]:
repeated_names

['data',
 'image',
 'la',
 'databases',
 'scale',
 'nosql',
 'algorithm',
 'http',
 'value',
 'database',
 'cluster',
 'al',
 'en',
 'eduroam',
 'space',
 'using',
 'number',
 'clustering',
 'used',
 'set',
 'vol',
 'key',
 'document',
 'processing',
 'based',
 'mean',
 'time',
 'server',
 'computer',
 'radius',
 'user',
 'means',
 'applications',
 'new',
 'complex',
 'order',
 'like',
 'real',
 'results',
 'equations',
 'www',
 'case',
 'access',
 'point',
 'relational',
 'et',
 'process',
 'let',
 'com',
 'use',
 'model',
 'ieee',
 'operations',
 'local',
 'information',
 'images',
 'structure',
 'analysis',
 'org',
 'function',
 'problem',
 'linear',
 'given',
 'systems',
 'documents',
 'group',
 'version',
 'object',
 'following',
 'theorem',
 'distributed',
 'methods',
 'clusters',
 'vector',
 'initial',
 'values',
 'network',
 'store',
 'query',
 'objects',
 'method',
 'level',
 'functions',
 'column',
 'client',
 'performance',
 'servers',
 'paper',
 'fig',
 'example',
 'solutio

# Gensim

En esta sección vamos a desarrollar el primer apartado opcional de la práctica, correspondiente a la extracción y visualización de tópicos usando la librería Gensim. Para realizar este ejercicio hemos seguido las instrucciones dadas en https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/. 

Tras realizar la importación de todas las librerías necesarias al comienzo de este notebook, el primer paso que debemos realizar es obtener las *stopwords* que se van a aplicar de la librería *nltk*, y obtener una lista que contendrá los documentos que vamos a usar. Como anteriormente ya hemos creado la lista articles, que añade el abstract de los documentos que no tienen el texto disponible, y el título de los documentos que no tienen el abstract disponible, la podemos reutilizar.

In [11]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
data = articles

Una vez obtenida la lista con la que vamos a aplicar gensim, ejecutamos la función *sent_to_words* sobre ella con dos motivos:
1. Filtrar la lista eliminando caracteres que pueden distraer al algoritmo, como son emails, caracteres de nueva línea y comillas simples.
2. Dividir cada elemento de la lista (cada artículo) en una lista de palabras, eliminando símbolos de puntuación con el parámetro deacc=True.

A continuación se construyen los modelos *bigram* y *trigram* que nos permiten extraer pares y tríos de palabras que aparecen frecuentemente juntas. Para ello, se utiliza la clase *Phrase*, que se puede observar en más detalle en https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases, y que detecta frases basadas en el *count* de la colocación de las palabras. El parámetro *min_count* que se le pasa a *Phrase* indica que como mínimo ese frase debe aparecer 5 veces en el texto, mientras que el parámetro *threshold* representa un umbral para escoger las frases que tengan una puntuación mayor de ese valor. Por defecto el sistema de puntuación que se elige se calcula de la sigueinte manera, obtenido de https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.original_scorer:

$$ \frac{(bigram\_count - min\_count)*len\_vocab}{worda\_count*wordb\_count},$$

siendo *bigram_count* el número de veces que un par *worda_wordb* aparece en la colección.

Por último, pasamos la salida obtenida aplicando *Phrases* a la función *Phraser*, que permite reducir el consumo de memoria de *Phrases*.

In [12]:
def sent_to_words(data):
    for sent in data:
        sent = re.sub(r'\S*@\S*\s?', '', sent)  # eliminamos emails
        sent = re.sub(r'\s+', ' ', sent)  # eliminamos caracteres de nueva línea
        sent = re.sub("\'", "", sent)  # eliminamos comillas simples
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

data_words = list(sent_to_words(data))

# construimos los modelos bigram y trigram
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


Una vez tenemos los modelos *bigram* y *trigram*, creamos a continuación funciones para aplicarlos. Además, creamos una función para eliminar las *stopwords*, y para aplicar *lemmatization*, el cual es un proceso que consiste en transformar cada palabra a su forma raíz, quedándonos únicamente con nombres, adjetivos, verbos y adverbios.

In [13]:
#funciones para stopwords, bigrams, trigrams y lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Definidas las funciones anteriores, las aplicamos a nuestras palabras extraídas de los artículos. Para aplicar la *lemmatization* usamos la librería *spacy*, el cual es un motor NLP que contiene modelos pre construidos que pueden *parsear* texto y computar varias características de NLP utilizando una sola llamada a una función. Con esta librería podemos extraer por tanto la forma raíz de las palabras.

In [14]:
# eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

# creamos los bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# creamos los trigrams
data_words_trigrams = make_trigrams(data_words_bigrams)

# inicializamos el modelo inglés ('en') de spacy
# para descargar: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# aplicamos lemmatization
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

El paso más importante a la hora de aplicar Gensim para extraer los tópicos de nuestra colección de artículos consiste en aplicar el algoritmo LDA (Latent Dirichlet Allocation), el cual representa los documentos como mezclas de tópicos en los que aparecen palabras con ciertas probabilidades. El proceso de entrenamiento de LDA se puede resumir como sigue:

1. Se fija el número de tópicos $K$ que se quiere generar, y de manera aleatoria se le asigna a cada palabra de cada documento un tópico aleatorio.
2. Para cada documento d <br>
   2.1 Para cada palabra w en d, se calculan dos probabilidades: P( tópico t | d) = la proporción de palabras en el documento d asignadas al tópico t, y P( w | t) = la proporción de asignaciones al tópico t a lo largo de todos los documentos que vienen de esta palabra w. Una vez calculadas, asignamos a la palabra w un nuevo tópico, el que cumpla que el producto $P(t|d) * P(w|t)$ sea mayor.
3. Una vez repetido el proceso un número considerable de veces, se llegará a un estado en el que las asignaciones serán buenas. 
4. Por último, se utilizan estas asignaciones para estimar las mezclas de tópicos de cada documento (mediante el conteo de la proporción de palabras asignadas a cada tópico en ese documento) y las palabras asociadas a cada tópico (contando la proporción de palabras asignadas a cada tópico en total).

Este proceso hace que se obtenga el número de tópicos $K$ relacionados que esperábamos. Esta información ha sido obtenida de http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/.

Antes de generar el modelo LDA, necesitamos obtener los dos parámetros de entrada principales del algoritmo: el diccionario y el corpus. El primero de ellos simplemente consiste en transformar los documentos a una representación de diccionario, mientras que el segundo consiste en una transformación de los documentos a una forma vectorizada, computando la frecuencia de cada palabra, incluyendo los bigrams y trigrams.

A continuación obtenemos esos parámetros y mostramos un ejemplo con la frecuencia de cada término para el primero de los documentos, donde podemos ver, al haber pocas palabras, que a ese documento no pudimos extraerle el texto y se ha procesado su abstract. De hecho, comprobando ese artículo en el fichero pubs.json, se ha observado que el abstract no contiene la palabra eduroam, sino que está contenida en el título, y por eso esta palabra no aparece en las frecuencias que hemos mostrado.

In [15]:
# creamos el diccionario
id2word = corpora.Dictionary(data_lemmatized)

# creamos el corpus con la frecuencia de cada término en los documentos
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]


# ejemplo en el que mostramos, la frecuencia de cada término para un determinado documento
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('coefficient', 1),
  ('constant', 1),
  ('correspond', 1),
  ('dimensional', 1),
  ('first', 1),
  ('follow', 1),
  ('generalize', 1),
  ('group', 1),
  ('irreducible', 1),
  ('natural', 1),
  ('order', 1),
  ('paper', 1),
  ('partial_differential_equation', 1),
  ('purpose', 1),
  ('representation', 1),
  ('rotation', 1),
  ('show', 1),
  ('system', 1),
  ('way', 1)]]

Una vez obtenidos los parámetros creamos el modelo LDA mencionado anteriormente. Si imprimimos los tópicos que devuelve el modelo, podemos observar cómo cada uno de ellos se ha obtenido como una combinación lineal de varias palabras.

In [16]:
# construimos el modelo LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


# imprimimos los 5 tópicos devueltos por el modelo LDA
pprint(lda_model.print_topics())

[(0,
  '0.028*"image" + 0.009*"pyramid" + 0.006*"processing" + 0.004*"level" + '
  '0.004*"digital" + 0.004*"scale" + 0.003*"fig" + 0.003*"process" + '
  '0.003*"may" + 0.003*"device"'),
 (1,
  '0.016*"datum" + 0.015*"database" + 0.010*"document" + 0.010*"node" + '
  '0.009*"nosql" + 0.008*"store" + 0.007*"use" + 0.007*"system" + '
  '0.006*"operation" + 0.005*"application"'),
 (2,
  '0.035*"cluster" + 0.033*"cid" + 0.023*"algorithm" + 0.020*"datum" + '
  '0.020*"mean" + 0.011*"use" + 0.008*"point" + 0.008*"image" + 0.008*"method" '
  '+ 0.008*"value"'),
 (3,
  '0.018*"user" + 0.015*"eduroam" + 0.011*"radius" + 0.007*"server" + '
  '0.007*"access" + 0.005*"network" + 0.005*"use" + 0.005*"attribute" + '
  '0.005*"institution" + 0.005*"service"'),
 (4,
  '0.018*"cid" + 0.010*"image" + 0.004*"equation" + 0.004*"give" + '
  '0.004*"function" + 0.003*"la" + 0.003*"para" + 0.003*"see" + '
  '0.003*"scale_space" + 0.003*"structure"')]


Para ver cómo de bueno es nuestro modelo, calculamos las medidas de perplejidad y de puntuación de coherencia. La primera de ellas se basa en el método probabilístico de medir la log-verosimilitud (logaritmo de la función de verosimilitud de una muestra aleatoria simple) de un conjunto de datos reservado para test. Por otro lado, el *coherence score* de un tópico se calcula como la media de las distancias entre las palabras.

A continuación mostramos las medidas de perplejidad y coherencia en nuestro caso:

In [17]:
# computamos perplejidad
print('\nPerplejidad: ', lda_model.log_perplexity(corpus))

# computamos coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplejidad:  -7.922907082785187

Coherence Score:  0.5361713723377118


Para visualizar los resultados obtenidos del modelo LDA anterior, utilizamos la librería pyLDAvis, que nos permite obtener un gráfico interactivo diseñado para que funcione bien con jupyter notebooks. 

Como se puede observar, el algoritmo ha conseguido separar bien los 5 tópicos, con excepción de los grupos 1 y 5 en los que hay palabras en común, quizás porque el procesamiento de imágenes (más presente en el grupo 5) tiene una componente matemática que se ha podido ligar a las principales palabras de las ecuaciones de Cauchy-Riemann (más presentes en el grupo 1). En general, en todos los grupos la gran mayoría de palabras se corresponden con un único tópico de los 5 que hemos introducido en la búsqueda de artículos al comienzo de la práctica.

In [18]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/rafa/anaconda3/envs/new_environment/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.147591  0.025494       1        1  37.782906
1     -0.201207 -0.102645       2        1  35.191719
2     -0.093706  0.191510       3        1  14.819378
3      0.052773 -0.103943       4        1   9.025909
0      0.094549 -0.010415       5        1   3.180090, topic_info=      Category         Freq            Term        Total  loglift  logprob
term                                                                      
3728   Default  1466.000000             cid  1466.000000  30.0000  30.0000
723    Default   762.000000           image   762.000000  29.0000  29.0000
10818  Default   746.000000         cluster   746.000000  28.0000  28.0000
2027   Default  1116.000000           datum  1116.000000  27.0000  27.0000
2906   Default   437.000000       algorithm   437.000000  26.0000  26.0000
954    Default   436.000000            mean   436.000000  25.0000  25.0000
3788   Default   308.000000            user   308.000000  24.0000  24.0000
3851   Default   709.000000        database   709.000000  23.0000  23.0000
7870   Default   252.000000         eduroam   252.000000  22.0000  22.0000
4941   Default   442.000000            node   442.000000  21.0000  21.0000
12077  Default   415.000000           nosql   415.000000  20.0000  20.0000
7057   Default   165.000000          radius   165.000000  19.0000  19.0000
7937   Default   289.000000          server   289.000000  18.0000  18.0000
439    Default   491.000000        document   491.000000  17.0000  17.0000
1602   Default   692.000000             use   692.000000  16.0000  16.0000
5085   Default   346.000000           store   346.000000  15.0000  15.0000
3557   Default   149.000000      processing   149.000000  14.0000  14.0000
1092   Default   289.000000       operation   289.000000  13.0000  13.0000
961    Default   293.000000          method   293.000000  12.0000  12.0000
1132   Default   330.000000           point   330.000000  11.0000  11.0000
3932   Default   181.000000           scale   181.000000  10.0000  10.0000
3608   Default   166.000000          access   166.000000   9.0000   9.0000
1611   Default   390.000000           value   390.000000   8.0000   8.0000
1755   Default   124.000000        distance   124.000000   7.0000   7.0000
1040   Default   328.000000          number   328.000000   6.0000   6.0000
1044   Default   312.000000          object   312.000000   5.0000   5.0000
13360  Default   203.000000         mongodb   203.000000   4.0000   4.0000
4707   Default   171.000000       attribute   171.000000   3.0000   3.0000
7938   Default   153.000000         service   153.000000   2.0000   2.0000
921    Default   138.000000        manifold   138.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
4787    Topic5    11.111303          device    35.592361   2.2841  -5.8867
3557    Topic5    24.395628      processing   149.527908   1.6352  -5.1003
3568    Topic5     9.555403          sample    32.682304   2.2185  -6.0376
4892    Topic5    17.598022           level   104.760735   1.6644  -5.4269
3471    Topic5     7.640598     compression    25.049656   2.2609  -6.2612
633     Topic5     5.378384              gl    13.290302   2.5436  -6.6123
3588    Topic5     6.282242          visual    19.752369   2.3027  -6.4570
3932    Topic5    17.128763           scale   181.635544   1.0870  -5.4539
5083    Topic5     6.126547           stage    20.743204   2.2287  -6.4821
558     Topic5    13.139106             fig   125.817268   1.1890  -5.7191
2232    Topic5    10.467075        original    76.881927   1.4542  -5.9465
1840    Topic5     9.688553         pattern    87.036530   1.2529  -6.0237
66      Topic5    10.555387        analysis   166.160843   0.6919  -5.9381
2762    Topic5    12.405853         process   320.709656   0.1959  -5.7765
952     Topic5    11.138795             may   296.3966

# Whoosh

El último apartado de la tarea consiste en realizar el ejercicio opcional sobre Whoosh. La información ha sido extraída de https://whoosh.readthedocs.io/en/latest/intro.html. Lo primero que hemos de realizar para poder indexar los documentos y hacer búsquedas sobre los índices es definir un Schema. Los schemas especifican los campos de los documentos en los índices.

Todos los atributos del Schema son de tipo TEXT, que como su propio nombre indica, sirve para representar texto escrito. El parámetro *stored* es un booleano que indica si el campo de texto estará disponible en los resultados de la búsqueda. Por otro lado, el atributo analyzer indica el tipo de analizador que permite extraer *tokens* de nuestro schema. Un analizador empaqueta un *tokenizador* y cero o más filtros en una única unidad.

En nuestro caso, hemos usado el analizador estándar, que consiste en un *tokenizador* de expresiones regulares con un filtro que transforma las palabras a minúscula, y un filtro que marca y elimina las *stopwords*.

In [19]:
# creamos el schema
schema = Schema(title=TEXT(stored=True),
                author=TEXT(stored=True),
                text=TEXT(stored=False, analyzer=StandardAnalyzer()),
                abstract=TEXT(stored=False, analyzer=StandardAnalyzer()))

Tras definir el esquema, el siguiente paso es añadir un índice a un directorio, usando ese esquema. En nuestro caso, hemos elegido el directorio actual. Para abrir ese índice en el directorio se usa la función *open_dir*. 

Una vez creado el índice, ya podemos añadirle documentos con un objeto IndexWriter. Para ello, obtenemos un dataframe leyendo el archivo json guardado en el paso 3 y extraemos las características de cada documento (definidas en el schema) para incorporarlas al IndexWriter. Cuando ya hemos terminado de añadir documentos, se llama a la función *commit*, que termina la escritura y desbloquea el índice.

In [20]:
# creamos el índice
ix = index.create_in(".", schema)
ix = index.open_dir(".")

writer = ix.writer()

# obtenemos el dataframe y añadimos al writer todos los documentos
df = pd.read_json('pubs.json')
for idx, row in df.iterrows():
    writer.add_document(title=row["title"], author=row["author"],
                   abstract=row["abstract"], text=row["text"])
writer.commit()

El paso final en este ejercicio es la realización de búsquedas sobre el índice anterior. Para ello se usa un objeto de tipo Searcher. El método más importante de Searcher es search(), que nos permite, dado un objeto de tipo Query, devolver el resultado de la búsqueda que le pidamos. 

En nuestro caso, para crear la query hemos de definir primero un objeto QueryParser, que recibe como parámetros el atributo del schema (el texto del documento) sobre el que queremos hacer la búsqueda, y el schema. Una vez lo tenemos, parseamos la frase que queremos buscar usando el parser y obtenemos nuestro objeto Query. 

Por último, abrimos el *searcher* para realizar la búsqueda, y definimos, con el parámetro *limit* de search(), que se devuelvan todos los documentos que hagan *match* con la query, sin límite. Cuando se completa la búsqueda, imprimimos todos los resultados obtenidos (en nuestro caso 3).

Cabe destacar que el objeto devuelto por el método search() es de tipo Results, el cual, tal y como dice la documentación (https://whoosh.readthedocs.io/en/latest/api/searching.html#whoosh.searching.Results), puede ser tratado como una lista de diccionarios, donde cada diccionario contiene los campos guardados (stored=True) al definir el schema.

Una vez hemos acabado, la función *with* se encarga de cerrar automáticamente el searcher.

In [21]:
qp = QueryParser("text", schema=ix.schema)
q = qp.parse(u"artificial intelligence")

with ix.searcher() as s:
    results = s.search(q, limit = None) # poniendo limit a None nos devuelve todos los resultados
    for result in results:
        print(result)


<Hit {'author': 'MM Rahman and DN Davis', 'title': 'Fuzzy unordered rules induction algorithm used as missing value imputation methods for k-mean clustering on real cardiovascular data'}>
<Hit {'author': 'B Decker and M Politze and R Renner', 'title': 'Device specific credentials to protect from identity theft in Eduroam'}>
<Hit {'author': 'J Yadav and M Sharma', 'title': 'A Review of K-mean Algorithm'}>
